In [1]:
import tensorflow as tf
from tensorflow.python.client import device_lib

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
device_lib.list_local_devices()

import os
import numpy as np
import matplotlib.pyplot as plt

#seed
from numpy.random import seed
seed(1)
#tf.random.set_seed(1745093)

#model
from tensorflow.keras import Sequential
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout, Conv2D, Flatten, LSTM, Reshape
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import LearningRateScheduler
from sklearn.metrics import classification_report

Num GPUs Available:  1


In [2]:
#dataset import 
X_train = np.load('X_train.npy')
Y_train = np.load('Y_train.npy')
X_test = np.load('X_test.npy')
Y_test = np.load('Y_test.npy')

In [3]:
#check for duplicates between x_train and x_test
lista5 = []
count5 = 0
for i in range (len(X_train)):
    bambino9 = X_train[i]
    for j in range (len(X_test)):
        if i<j :
            bambino10 = X_test[j]
            flag = np.array_equal(bambino9, bambino10)
            if (flag == True) and (i not in lista5):
                count5+=1
                lista5.append(i)
print(count5)
print(lista5)

0
[]


In [4]:
#dataset shuffle 
from sklearn.utils import shuffle

X_train, Y_train = shuffle(X_train, Y_train)
X_test, Y_test = shuffle(X_test, Y_test)
X_shape_sample = np.shape(X_train[0])
avg_len = len(X_train[0])
              
print(np.shape(X_train))
print(np.shape(Y_train))
print(np.shape(X_test))
print(np.shape(Y_test))
print(avg_len)

(565, 35, 75)
(565, 2)
(143, 35, 75)
(143, 2)
35


In [5]:
#MLP IMPLEMENTATION 
#reshaping x_train from (565, 35, 75) to (565, 2625)
x_train = []
for i in range (len(X_train)):
    sample = np.reshape(X_train[i], -1)
    x_train.append(sample)
x_train = np.asarray(x_train)

print(np.shape(x_train))
print(type(x_train))
input_shape = np.shape(x_train[0])
print(input_shape)

train_acc_list = []
train_time_list = []

#accuracy
def accur(y_test, y_test_pred):
    count = 0
    for i in range(np.shape(y_test)[0]):
        if y_test[i] == y_test_pred[i]:
            count+=1
    return count/np.shape(y_test)[0]*100

Y_train_b = np.argmax(Y_train, axis = -1)
import time
model = Sequential()
model.add(Dense(64, input_shape=input_shape, activation='tanh'))
model.add(Dense(32, activation='tanh'))
model.add(Dense(16, activation='tanh'))
model.add(Dense(2, activation = "softmax"))
model.summary()
early_stop = EarlyStopping(monitor='val_loss', patience=700, restore_best_weights = False)
opt =keras.optimizers.Adam(learning_rate=0.0001)
model.compile(loss='binary_crossentropy',optimizer=opt ,metrics=['accuracy'])
start = time.time()
history = model.fit(x_train, Y_train, epochs=300, batch_size=8, validation_split = 0.1 ,callbacks =[early_stop], verbose=0)
end = time.time()
print('time for training in seconds ',end-start)
train_time_list.append(end-start)
#prediction
#y_predict = np.argmax(model.predict(X_test), axis=-1)
y_predict_train = np.argmax(model.predict(x_train),axis = -1)
#accuracy = accur(Y_test_b, y_predict)
train_accuracy = accur(Y_train_b,y_predict_train)
train_acc_list.append(train_accuracy)
#plotting the confusion matrix
print('\ntraining confusion matrix \n ',tf.math.confusion_matrix(
Y_train_b, y_predict_train, num_classes=None, weights=None, dtype=tf.dtypes.int32,
name=None))
print(train_accuracy)

(565, 2625)
<class 'numpy.ndarray'>
(2625,)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                168064    
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 16)                528       
                                                                 
 dense_3 (Dense)             (None, 2)                 34        
                                                                 
Total params: 170,706
Trainable params: 170,706
Non-trainable params: 0
_________________________________________________________________
time for training in seconds  78.30720138549805

training confusion matrix 
  tf.Tensor(
[[260   8]
 [  1 296]], shape=(2, 2), dtype=int32